# Model Training

In [16]:
import pandas as pd

In [17]:
df=pd.read_csv('./data/gemstone.csv')

In [18]:
df=df.drop(labels=['id'],axis=1)

In [19]:
#Independent and Dependent Vaiable
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [20]:
categorical_cols=X.columns[X.dtypes=='O']
numerical_cols=X.columns[X.dtypes!='O']

In [21]:
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [22]:
from sklearn.impute import SimpleImputer # This is used to handle the missing values
from sklearn.preprocessing import StandardScaler # This is for Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding
#pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [23]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
## Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

In [24]:
preprocessor=ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numerical_cols),
        ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [26]:
preprocessor.fit_transform(X_train)

array([[-0.82314374, -1.12998781, -0.64189666, ...,  0.87410007,
        -0.93674681,  1.35074594],
       [ 0.94502267, -1.77782269,  0.92190185, ..., -1.13764403,
         0.91085333,  0.68445511],
       [ 1.9584839 ,  0.16568195,  0.40063568, ..., -0.13177198,
         0.91085333,  0.01816428],
       ...,
       [ 0.92345966,  0.90606467,  0.40063568, ..., -0.13177198,
         0.29498662,  0.01816428],
       [-1.03877378, -0.66724861, -0.64189666, ..., -1.13764403,
         0.29498662,  2.01703677],
       [-1.03877378, -0.01941373,  0.92190185, ..., -1.13764403,
         0.29498662, -1.31441737]])

In [30]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.fit_transform(X_test),columns=preprocessor.get_feature_names_out())

In [31]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


In [58]:
# Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [34]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [35]:
regression.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
          652.10059539]])

In [36]:
regression.intercept_

array([3976.8787389])

In [61]:
import numpy as np
def evaulate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square,mse

In [63]:
## Train Multiple Models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    #Make Prediction
    y_pred=model.predict(X_test)
    mae,rmse,r2_square,mse=evaulate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    r2_list.append(r2_square)
    print("Model Training Performance")
    print("RMSE:",rmse)
    print("MSE:",mse)
    print("MAE:",mae)
    print("R2 Score:",r2_square)
    

LinearRegression
Model Training Performance
RMSE: 1015.0853051770308
MSE: 1030398.1767863458
MAE: 678.6599280042764
R2 Score: 0.9362334489637519
Lasso
Model Training Performance
RMSE: 1015.0659567672999
MSE: 1030358.896587914
MAE: 679.9362394438181
R2 Score: 0.9362358798325504
Ridge
Model Training Performance
RMSE: 1015.0875649237839
MSE: 1030402.7644628973
MAE: 678.6934635146374
R2 Score: 0.9362331650537863
ElasticNet
Model Training Performance
RMSE: 1526.1838883233627
MSE: 2329237.260977818
MAE: 1065.7688041796291
R2 Score: 0.855854338619943


In [53]:
X_test

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.626368,0.258533,-0.114042,-0.597859,-0.578700,-0.572348,0.870276,-1.544714,-0.652004
1,2.633302,-2.139001,-0.114042,2.144241,2.217298,1.988647,-1.141533,0.305387,-1.316935
2,-1.126185,-1.216872,0.927721,-1.376145,-1.416589,-1.479672,-0.135629,-0.928014,2.007720
3,-1.017529,-0.571382,0.927721,-1.158949,-1.161580,-1.201621,-0.135629,1.538788,2.007720
4,0.873080,0.627385,-0.634923,0.958713,0.996895,1.022786,0.870276,0.922088,-0.652004
...,...,...,...,...,...,...,...,...,...
58067,0.264608,0.535172,0.927721,0.424772,0.377586,0.437416,-1.141533,1.538788,-1.316935
58068,-0.604637,0.535172,-0.634923,-0.525460,-0.551378,-0.499177,0.870276,-1.544714,0.012927
58069,-0.821949,-0.018105,-0.634923,-0.833155,-0.861032,-0.850399,0.870276,0.305387,2.007720
58070,0.916542,-0.663595,1.448603,1.058261,0.978680,0.949615,-0.135629,1.538788,-0.652004


In [65]:
print(r2_list)
print(model_list)

[0.9362334489637519, 0.9362358798325504, 0.9362331650537863, 0.855854338619943]
['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']
